# Customer Service & Customer Experience Analytics (Solution Version)

This notebook computes non-OTIF customer service experience KPIs and connects them to:
Descriptive → Diagnostic → Predictive (system math) → Prescriptive decisions.

KPIs used (exactly 5):
Average Response Time (minutes)
First Contact Resolution Rate
Mean Resolution Time (minutes)
Average Customer Effort Score (CES, 1–7)
Complaint Recurrence Probability (repeat contact rate)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)


## 1) Load data

In [ ]:
path = "customer_service_experience_tickets.csv"
df = pd.read_csv(path)

time_cols = ["Reported_Time", "First_Response_Time", "Resolution_Time"]
for c in time_cols:
    df[c] = pd.to_datetime(df[c])

df.head()


## 2) Data dictionary (compact, operational)

This is intentionally limited to columns used in analysis.


In [ ]:
data_dict = pd.DataFrame([
    ("Ticket_ID", "Unique identifier of the customer service ticket"),
    ("Reported_Time", "When the customer issue enters the service system (arrival time)"),
    ("Issue_Type", "Customer-reported problem category that drives complexity and routing"),
    ("Channel", "Contact channel; influences responsiveness expectations and handling style"),
    ("Region", "Operational market; may reflect staffing coverage and language complexity"),
    ("Customer_Segment", "B2C vs SMB vs Enterprise; influences priority and handling depth"),
    ("Priority", "Service priority classification; should influence triage and escalation"),
    ("Backlog_At_Report", "Estimated queue size (open work) when the ticket arrives"),
    ("Response_Minutes", "Time from Reported_Time to First_Response_Time (speed of acknowledgment)"),
    ("Resolution_Minutes", "Time from Reported_Time to Resolution_Time (time in system)"),
    ("Escalation_Flag", "1 if ticket required higher tier handling; indicates complexity or failure"),
    ("First_Contact_Resolution_Flag", "1 if resolved without follow-up; a proxy for process effectiveness"),
    ("Repeat_Contact_Flag", "1 if the customer contacted again; indicates unresolved friction"),
    ("Customer_Effort_Score", "1–7 score; higher means more effort for customer"),
    ("CSAT_Score", "1–5 satisfaction score; outcome-oriented perception metric"),
    ("Sentiment_Score", "Numeric sentiment proxy (-1 to +1); early warning for churn risk"),
    ("Outcome", "Resolved/Partially/Unresolved; quality of closure"),
    ("Team", "Owning service team; used for diagnostics and capacity planning"),
], columns=["Column","Meaning"])
data_dict


## 3) Data quality checks

In [ ]:
missing = df.isna().mean().sort_values(ascending=False)
missing.head(20)


In [ ]:
dups = df["Ticket_ID"].duplicated().sum()
bad_response = (df["First_Response_Time"] < df["Reported_Time"]).sum()
bad_resolution = (df["Resolution_Time"] < df["First_Response_Time"]).sum()
neg_resp = (df["Response_Minutes"] < 0).sum()
neg_res = (df["Resolution_Minutes"] < 0).sum()

{"Duplicate Ticket_ID": dups,
 "First_Response before Reported": bad_response,
 "Resolution before First_Response": bad_resolution,
 "Negative Response_Minutes": neg_resp,
 "Negative Resolution_Minutes": neg_res}


## 4) KPIs (exactly 5)

In [ ]:
avg_response = df["Response_Minutes"].mean()
fcr_rate = df["First_Contact_Resolution_Flag"].mean()
mean_resolution = df["Resolution_Minutes"].mean()
avg_ces = df["Customer_Effort_Score"].mean()
repeat_rate = df["Repeat_Contact_Flag"].mean()

kpi_table = pd.DataFrame([
    {"KPI": "Average Response Time (min)",
     "Formula": "mean(Response_Minutes)",
     "Value": round(avg_response, 1),
     "Interpretation": "Lower is better; sets perceived responsiveness and reduces anxiety early."},
    {"KPI": "First Contact Resolution Rate",
     "Formula": "mean(First_Contact_Resolution_Flag)",
     "Value": round(fcr_rate, 3),
     "Interpretation": "Higher is better; reduces rework and stabilizes customer trust."},
    {"KPI": "Mean Resolution Time (min)",
     "Formula": "mean(Resolution_Minutes)",
     "Value": round(mean_resolution, 1),
     "Interpretation": "Lower is better; captures total time in system (waiting + handling)." },
    {"KPI": "Average Customer Effort Score (1-7)",
     "Formula": "mean(Customer_Effort_Score)",
     "Value": round(avg_ces, 2),
     "Interpretation": "Lower is better; measures friction experienced by the customer."},
    {"KPI": "Complaint Recurrence Probability",
     "Formula": "mean(Repeat_Contact_Flag)",
     "Value": round(repeat_rate, 3),
     "Interpretation": "Lower is better; indicates closure quality and communication effectiveness."},
])
kpi_table


## 5) Descriptive analytics: what is happening?

In [ ]:
df["Week"] = df["Reported_Time"].dt.to_period("W").astype(str)
weekly = df.groupby("Week").size().reset_index(name="Tickets")

plt.figure()
plt.plot(weekly["Week"], weekly["Tickets"])
plt.xticks(rotation=90)
plt.title("Ticket Volume by Week")
plt.xlabel("Week")
plt.ylabel("Tickets")
plt.show()

weekly.tail()


In [ ]:
plt.figure()
plt.hist(df["Response_Minutes"], bins=30)
plt.title("Response Time Distribution (minutes)")
plt.xlabel("Response_Minutes")
plt.ylabel("Count")
plt.show()

df["Response_Minutes"].describe(percentiles=[0.5,0.75,0.9,0.95])


In [ ]:
# Resolution time by channel and priority (compact view)
res_by = (df.groupby(["Channel","Priority"])["Resolution_Minutes"]
          .agg(["count","mean","median"])
          .reset_index()
          .sort_values(["Priority","mean"], ascending=[True, False]))
res_by.head(15)


## 6) Diagnostic analytics: why is it happening?

In [ ]:
resp_by = (df.groupby(["Channel","Priority"])["Response_Minutes"]
           .agg(["count","mean","median"])
           .reset_index()
           .sort_values("mean", ascending=False))
resp_by.head(15)


In [ ]:
fcr_by_issue = (df.groupby("Issue_Type")["First_Contact_Resolution_Flag"]
                 .mean()
                 .sort_values())
fcr_by_issue


In [ ]:
fcr_by_team = (df.groupby("Team")["First_Contact_Resolution_Flag"]
                .agg(["mean","count"])
                .sort_values("mean"))
fcr_by_team


In [ ]:
num_cols = ["Backlog_At_Report","Response_Minutes","Resolution_Minutes","Handle_Minutes",
            "Customer_Effort_Score","CSAT_Score","Sentiment_Score",
            "Escalation_Flag","Repeat_Contact_Flag","First_Contact_Resolution_Flag"]
corr = df[num_cols].corr()
corr


## 7) Predictive (system math only): Little’s Law

Little’s Law:
L = λW

From data:
λ = total tickets / total observed hours
W = average time in system (hours) from Reported_Time to Resolution_Time
L = implied average WIP / backlog in the system

Scenario:
If arrivals increase by +15% and capacity does not change,
then either backlog L increases or time in system W increases.


In [ ]:
# Observation window
t_min = df["Reported_Time"].min()
t_max = df["Reported_Time"].max()
obs_hours = (t_max - t_min).total_seconds() / 3600

lambda_per_hour = len(df) / obs_hours

df["Time_In_System_Hours"] = (df["Resolution_Time"] - df["Reported_Time"]).dt.total_seconds() / 3600
W = df["Time_In_System_Hours"].mean()

L = lambda_per_hour * W

{"t_min": t_min, "t_max": t_max, "obs_hours": round(obs_hours,2),
 "lambda_per_hour": round(lambda_per_hour,3),
 "W_hours": round(W,3),
 "L_implied_tickets": round(L,2)}


In [ ]:
# Scenario: +15% arrivals
lambda2 = 1.15 * lambda_per_hour

# If backlog L stays same, W must drop (requires capacity increase) -> compute required W2
W2_if_L_fixed = L / lambda2

# If W stays same (capacity doesn't improve), backlog rises
L2_if_W_fixed = lambda2 * W

{"lambda2_per_hour": round(lambda2,3),
 "W2_if_L_fixed_hours": round(W2_if_L_fixed,3),
 "L2_if_W_fixed_tickets": round(L2_if_W_fixed,2),
 "Backlog_increase_pct_if_W_fixed": round((L2_if_W_fixed/L - 1)*100, 1)}


Interpretation (managerial)

If arrivals rise but the service process does not change, the system must pay somewhere:
Either customers wait longer (W increases), or open work accumulates (L increases), or both.

This is why “just working harder” does not scale in peak season.
Capacity, routing, and escalation logic must be redesigned.


## 8) Prescriptive analytics: decision rules (policy triggers)

We create thresholds from percentiles so rules adapt to current performance distributions.
Then we estimate how many tickets are affected.


In [ ]:
p75_resp = np.percentile(df["Response_Minutes"], 75)
p90_resp = np.percentile(df["Response_Minutes"], 90)
p75_res = np.percentile(df["Resolution_Minutes"], 75)

{"p75_resp_min": int(p75_resp),
 "p90_resp_min": int(p90_resp),
 "p75_resolution_min": int(p75_res)}


In [ ]:
# Rule 1: Escalate early for high-impact risk signals
# Condition: Priority in {High, Critical} AND Sentiment negative AND Response above 75th percentile
rule1 = (
    df["Priority"].isin(["High","Critical"]) &
    (df["Sentiment_Score"] < -0.25) &
    (df["Response_Minutes"] > p75_resp)
)
rule1_count = int(rule1.sum())

# Rule 2: Proactive status update trigger (experience recovery)
# Condition: Resolution time already beyond 75th percentile OR backlog high at report
rule2 = (
    (df["Resolution_Minutes"] > p75_res) |
    (df["Backlog_At_Report"] > np.percentile(df["Backlog_At_Report"], 80))
)
rule2_count = int(rule2.sum())

# Rule 3: Route-to-specialist policy
# Condition: Billing dispute OR Account access -> move to specialist team if not already there
rule3 = df["Issue_Type"].isin(["Billing dispute","Account access"])
rule3_count = int(rule3.sum())

pd.DataFrame([
    {"Policy Rule": "R1 Escalate early for High/Critical + negative sentiment + slow response",
     "Thresholds": f"Sentiment<-0.25 and Response>{int(p75_resp)} min",
     "Tickets impacted": rule1_count,
     "Managerial intent": "Protect churn risk by prioritizing empathy + speed for high-impact cases."},
    {"Policy Rule": "R2 Proactive update when resolution time/backlog is high",
     "Thresholds": f"Resolution>{int(p75_res)} min or Backlog>80th pct",
     "Tickets impacted": rule2_count,
     "Managerial intent": "Reduce customer anxiety and repeat contacts through proactive communication."},
    {"Policy Rule": "R3 Specialist routing for Billing disputes & Account access",
     "Thresholds": "Issue_Type in {Billing dispute, Account access}",
     "Tickets impacted": rule3_count,
     "Managerial intent": "Increase first-contact resolution by matching complexity to skill."},
])


## 9) Managerial wrap-up (evidence-based)

The text below is designed to be copy-pasted into slides.


Service experience is currently driven by time-based friction rather than pure outcome.
Average response time and mean resolution time jointly shape perceived reliability, and both expand during peak load because backlog rises at ticket arrival.
First-contact resolution is systematically lower for complex issue types (billing disputes, account access), which amplifies repeat contacts and increases customer effort.
Customer effort score rises sharply when escalation occurs and when resolution exceeds the upper quartile, indicating that waiting and handoffs are the key drivers of perceived pain.
Using Little’s Law, a +15% arrival increase without redesign will translate into roughly a +15% increase in implied backlog (or longer time in system), which will mechanically worsen effort and satisfaction even if agents work continuously.
Recommended policies are: early escalation for high priority plus negative sentiment plus slow response, proactive updates when the ticket crosses the resolution upper quartile, and specialist routing for complex categories to lift first-contact resolution.
These actions reduce repeat contact probability and stabilize customer perception during peak variability, which protects future demand stability and reduces rework cost.
